# SPH

Create Your Own Smoothed-Particle-Hydrodynamics Simulation (With Python)
Philip Mocz (2020) Princeton Univeristy, @PMocz

Simulate the structure of a star with SPH

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import gamma

---

### Gausssian Smoothing kernel (3D)

- x     is a vector/matrix of x positions
- y     is a vector/matrix of y positions
- z     is a vector/matrix of z positions
- h     is the smoothing length
- w     is the evaluated smoothing function

In [2]:
def W_gaussian(x, y, z, h):
    r = np.sqrt(x**2 + y**2 + z**2)

    w = (1.0 / (h * np.sqrt(np.pi))) ** 3 * np.exp(-(r**2) / h**2)
    return w

---

### Gradient of the Gausssian Smoothing kernel (3D)

- x     is a vector/matrix of x positions
- y     is a vector/matrix of y positions
- z     is a vector/matrix of z positions
- h     is the smoothing length
- wx, wy, wz     is the evaluated gradient

In [3]:
def grad_W_gaussian(x, y, z, h):
    r = np.sqrt(x**2 + y**2 + z**2)

    n = -2 * np.exp(-(r**2) / h**2) / h**5 / (np.pi) ** (3 / 2)
    wx = n * x
    wy = n * y
    wz = n * z

    return wx, wy, wz

---

### Get pairwise desprations between 2 sets of coordinates

- ri    is an M x 3 matrix of positions
- rj    is an N x 3 matrix of positions
- dx, dy, dz   are M x N matrices of separations

In [4]:
def getPairwiseSeparations(ri, rj):

    M = ri.shape[0]
    N = rj.shape[0]

    # positions ri = (x,y,z)
    rix = ri[:, 0].reshape((M, 1))
    riy = ri[:, 1].reshape((M, 1))
    riz = ri[:, 2].reshape((M, 1))

    # other set of points positions rj = (x,y,z)
    rjx = rj[:, 0].reshape((N, 1))
    rjy = rj[:, 1].reshape((N, 1))
    rjz = rj[:, 2].reshape((N, 1))

    # matrices that store all pairwise particle separations: r_i - r_j
    dx = rix - rjx.T
    dy = riy - rjy.T
    dz = riz - rjz.T

    return dx, dy, dz

---

### Get Density at sampling loctions from SPH particle distribution

- r     is an M x 3 matrix of sampling locations
- pos   is an N x 3 matrix of SPH particle positions
- m     is the particle mass
- h     is the smoothing length
- rho   is M x 1 vector of densities

In [5]:
def getDensity(r, pos, m, h, W):
    M = r.shape[0]
    dx, dy, dz = getPairwiseSeparations(r, pos)
    rho = np.sum(m * W(dx, dy, dz, h), 1).reshape((M, 1))
    return rho

---

### Equation of State

- rho   vector of densities
- k     equation of state constant
- n     polytropic index
- P     pressure

In [6]:
def getPressure(rho, k, n):
    P = k * rho ** (1 + 1 / n)
    return P

---

### Calculate the acceleration on each SPH particle

- pos   is an N x 3 matrix of positions
- vel   is an N x 3 matrix of velocities
- m     is the particle mass
- h     is the smoothing length
- k     equation of state constant
- n     polytropic index
- mbda external force constant
- nu    viscosity
- a     is N x 3 matrix of accelerations

In [7]:
def getAcc(pos, vel, m, h, k, n, lmbda, nu, kernel, grad_kernel):
    N = pos.shape[0]

    # Calculate densities at the position of the particles
    rho = getDensity(pos, pos, m, h, kernel)

    # Get the pressures
    P = getPressure(rho, k, n)

    # Get pairwise distances and gradients
    dx, dy, dz = getPairwiseSeparations(pos, pos)
    dWx, dWy, dWz = grad_kernel(dx, dy, dz, h)

    # Add Pressure contribution to accelerations
    ax = -np.sum(m * (P / rho**2 + P.T / rho.T**2) * dWx, 1).reshape((N, 1))
    ay = -np.sum(m * (P / rho**2 + P.T / rho.T**2) * dWy, 1).reshape((N, 1))
    az = -np.sum(m * (P / rho**2 + P.T / rho.T**2) * dWz, 1).reshape((N, 1))

    # pack together the acceleration components
    a = np.hstack((ax, ay, az))

    # Add external potential force
    a -= lmbda * pos

    # Add viscosity
    a -= nu * vel

    return a

---

# SPH simulation

### Simulation plotting


In [8]:
def plot_system(rho, pos, lmbda, m, R, h, k, save_path, kernel):
    # prep figure
    fig = plt.figure(figsize=(4, 5), dpi=80)
    grid = plt.GridSpec(3, 1, wspace=0.0, hspace=0.3)
    ax1 = plt.subplot(grid[0:2, 0])
    ax2 = plt.subplot(grid[2, 0])
    rr = np.zeros((100, 3))
    rlin = np.linspace(0, 1, 100)
    rr[:, 0] = rlin
    rho_analytic = lmbda / (4 * k) * (R**2 - rlin**2)

    plt.sca(ax1)
    plt.cla()
    cval = np.minimum((rho - 3) / 3, 1).flatten()
    plt.scatter(pos[:, 0], pos[:, 1], c=cval, cmap=plt.cm.autumn, s=3, alpha=0.5)
    ax1.set(xlim=(-2.8, 2.8), ylim=(-2.4, 2.4))
    ax1.set_aspect("equal", "box")
    ax1.set_facecolor("black")
    ax1.set_facecolor((0.1, 0.1, 0.1))

    plt.sca(ax2)
    plt.cla()
    ax2.set(xlim=(0, 1), ylim=(0, 3))
    ax2.set_aspect(0.1)
    plt.plot(rlin, rho_analytic, color="gray", linewidth=2)
    rho_radial = getDensity(rr, pos, m, h, kernel)
    plt.plot(rlin, rho_radial, color="blue")
    plt.savefig(save_path, dpi=240)
    plt.close()

---

### Simulation Main Loop

In [9]:
from pathlib import Path
from tqdm import tqdm
import imageio.v2 as imageio


def run_simulation(
    plot_name,
    pos,
    vel,
    t,  # current time of the simulation
    tEnd,  # time at which simulation ends
    dt,  # timestep
    M,  # star mass
    R,  # star radius
    h,  # smoothing length
    k,  # equation of state constant
    n,  # polytropic index
    nu,  # damping
    kernel,
    grad_kernel,
    only_one=False,
):
    N = pos.shape[0]

    save_path = Path("plots")
    save_path.mkdir(exist_ok=True, parents=True)

    lmbda = (
        (2 * k * (1 + n))
        * (np.pi ** (-3 / (2 * n)))
        * (M * gamma(5 / 2 + n) / R**3 / gamma(1 + n)) ** (1 / n)
        / R**2
    )

    m = M / N  # single particle mass
    acc = getAcc(
        pos, vel, m, h, k, n, lmbda, nu, kernel, grad_kernel
    )  # calculate initial gravitational accelerations
    Nt = int(np.ceil(tEnd / dt))  # number of timesteps

    for i in tqdm(range(Nt)):
        if only_one:
            return None, None

        vel += acc * dt / 2  # (1/2) kick
        pos += vel * dt  # drift
        acc = getAcc(
            pos, vel, m, h, k, n, lmbda, nu, kernel, grad_kernel
        )  # update accelerations
        vel += acc * dt / 2  # (1/2) kick
        t += dt  # update time
        rho = getDensity(pos, pos, m, h, kernel)  # get density for plotting

        plot_system(
            rho, pos, lmbda, m, R, h, k, save_path / f"{plot_name}_{i}.png", kernel
        )

    ims = [imageio.imread(save_path / f"{plot_name}_{i}.png") for i in tqdm(range(Nt))]
    imageio.mimwrite(f"{plot_name}.gif", ims)

    return pos, vel

In [10]:
# set the random number generator seed
np.random.seed(42)

In [13]:
N = 400  # Number of particles
pos = np.random.randn(N, 3)
# vel = np.zeros(pos.shape)
vel = np.random.randn(N, 3)

pos, vel = run_simulation(
    "gaussian",
    pos,
    vel,
    t=0,  # current time of the simulation
    tEnd=12,  # time at which simulation ends
    dt=0.04,  # timestep
    M=2,  # star mass
    R=0.75,  # star radius
    h=0.1,  # smoothing length
    k=0.1,  # equation of state constant
    n=1,  # polytropic index
    nu=0.75,  # damping
    kernel=W_gaussian,
    grad_kernel=grad_W_gaussian,
)

100%|██████████| 300/300 [00:03<00:00, 81.02it/s]


In [14]:
pos2 = np.concatenate([pos + (1, 1, 0), pos + (-1, -1, 0)])
vel2 = np.concatenate([vel + (-3, -1, 0), vel + (3, 1, 0)])

pos2 += np.random.normal(size=pos2.shape) * 0.01
vel2 += np.random.normal(size=pos2.shape) * 0.1

pos3, vel3 = run_simulation(
    "gaussian_2_stars",
    pos2,
    vel2,
    t=0,  # current time of the simulation
    tEnd=24,  # time at which simulation ends
    dt=0.04,  # timestep
    M=3,  # star mass
    R=0.75,  # star radius
    h=0.1,  # smoothing length
    k=0.1,  # equation of state constant
    n=0.75,  # polytropic index
    nu=0.3,  # damping
    kernel=W_gaussian,
    grad_kernel=grad_W_gaussian,
)

100%|██████████| 600/600 [00:07<00:00, 81.60it/s]


Gaussian kernel used for simulating star collision allows is visualized in `gaussian_2_stars.gif` file.

A collision looks believable though at first it seems that the two stars pass through each other.

---

# Different kernels

### Cubic Spline Kernel

In [15]:
def cubic_spline(x, y, z, h):
    r = np.sqrt(x**2 + y**2 + z**2)
    q = r / h

    result = np.zeros(shape=r.shape)

    r_1h = (1 - 1.5 * q**2 + 0.75 * q**3) / (h**3 * np.pi)
    r_1h_mask = r <= h

    r_2h = (2 - q) ** 3 / (4 * h**3 * np.pi)
    r_2h_mask = (r > h) & (r <= 2 * h)

    result[r_1h_mask] = r_1h[r_1h_mask]
    result[r_2h_mask] = r_2h[r_2h_mask]

    return result


def cubic_spline_derivative(x, y, z, h):
    r = np.sqrt(x**2 + y**2 + z**2)
    q = r / h

    n = np.zeros(shape=r.shape)

    r_1h = (-12 * q + 9 * q**2) / (4 * h**4 * np.pi)
    r_1h_mask = r <= h

    r_2h = -3 * (2 - q) ** 2 / (4 * h**4 * np.pi)
    r_2h_mask = (r > h) & (r <= 2 * h)

    n[r_1h_mask] = r_1h[r_1h_mask]
    n[r_2h_mask] = r_2h[r_2h_mask]

    np.fill_diagonal(r, 1)

    wx = n * x / r
    wy = n * y / r
    wz = n * z / r

    return wx, wy, wz

In [16]:
N = 400  # Number of particles
pos = np.random.randn(N, 3)
# vel = np.zeros(pos.shape)
vel = np.random.randn(N, 3)

pos, vel = run_simulation(
    "cubic_spline",
    pos,
    vel,
    t=0,  # current time of the simulation
    tEnd=12,  # time at which simulation ends
    dt=0.04,  # timestep
    M=2,  # star mass
    R=0.75,  # star radius
    h=0.1,  # smoothing length
    k=0.1,  # equation of state constant
    n=1,  # polytropic index
    nu=1,  # damping
    kernel=cubic_spline,
    grad_kernel=cubic_spline_derivative,
)

100%|██████████| 300/300 [00:06<00:00, 47.78it/s]


In [17]:
pos2 = np.concatenate([pos + (1, 1, 0), pos + (-1, -1, 0)])
vel2 = np.concatenate([vel + (-3, -1, 0), vel + (3, 1, 0)])

pos2 += np.random.normal(size=pos2.shape) * 0.01
vel2 += np.random.normal(size=pos2.shape) * 0.1

pos3, vel3 = run_simulation(
    "cubic_spline_2_stars",
    pos2,
    vel2,
    t=0,  # current time of the simulation
    tEnd=24,  # time at which simulation ends
    dt=0.04,  # timestep
    M=3,  # star mass
    R=0.75,  # star radius
    h=0.1,  # smoothing length
    k=0.1,  # equation of state constant
    n=0.75,  # polytropic index
    nu=0.3,  # damping
    kernel=cubic_spline,
    grad_kernel=cubic_spline_derivative,
)

100%|██████████| 600/600 [00:07<00:00, 78.63it/s]


cubic spline allows for another decent simulation similar to gaussian kernel

Result in file `cubic_spline_2_stars.gif`

---

### Poly6 Kernel and its derivative

In [18]:
def poly6(x, y, z, h):
    r = np.sqrt(x**2 + y**2 + z**2)

    result = np.zeros(shape=r.shape)

    r_h = 315 * (h**2 - r**2) ** 3 / (64 * np.pi * h**9)
    r_h_mask = r <= h

    result[r_h_mask] = r_h[r_h_mask]

    return result


def poly6_derivative(x, y, z, h):
    r = np.sqrt(x**2 + y**2 + z**2)

    n = np.zeros(shape=r.shape)

    r_h = -6 * 315 * r * (h**2 - r**2) ** 2 / (64 * np.pi * h**9)
    r_h_mask = r <= h

    n[r_h_mask] = r_h[r_h_mask]

    np.fill_diagonal(r, 1)

    wx = n * x / r
    wy = n * y / r
    wz = n * z / r

    return wx, wy, wz

In [19]:
N = 400  # Number of particles
pos = np.random.randn(N, 3)
# vel = np.zeros(pos.shape)
vel = np.random.randn(N, 3)

pos, vel = run_simulation(
    "poly6",
    pos,
    vel,
    t=0,  # current time of the simulation
    tEnd=12,  # time at which simulation ends
    dt=0.04,  # timestep
    M=2,  # star mass
    R=0.75,  # star radius
    h=0.1,  # smoothing length
    k=0.1,  # equation of state constant
    n=1,  # polytropic index
    nu=1,  # damping
    kernel=poly6,
    grad_kernel=poly6_derivative,
)

100%|██████████| 300/300 [00:03<00:00, 77.54it/s]


In [20]:
pos2 = np.concatenate([pos + (1, 1, 0), pos + (-1, -1, 0)])
vel2 = np.concatenate([vel + (-3, -1, 0), vel + (3, 1, 0)])

pos2 += np.random.normal(size=pos2.shape) * 0.01
vel2 += np.random.normal(size=pos2.shape) * 0.1

pos3, vel3 = run_simulation(
    "poly6_2_stars",
    pos2,
    vel2,
    t=0,  # current time of the simulation
    tEnd=24,  # time at which simulation ends
    dt=0.04,  # timestep
    M=2,  # star mass
    R=0.75,  # star radius
    h=0.1,  # smoothing length
    k=0.1,  # equation of state constant
    n=0.75,  # polytropic index
    nu=1,  # damping
    kernel=poly6,
    grad_kernel=poly6_derivative,
)

100%|██████████| 600/600 [00:07<00:00, 79.03it/s]


Even though at first it seemed that using poly6 kernel doesn't seem to be affecting the density (rho), performing a star collision simulation (`poly6_2_stars.gif`) shows changes in brightness that signify higher function values

---

### Spiky Kernel and its derivative

In [21]:
def spiky(x, y, z, h):
    r = np.sqrt(x**2 + y**2 + z**2)

    result = np.zeros(shape=r.shape)

    r_h = 15 * (h - r) ** 3 / (np.pi * h**6)
    r_h_mask = r <= h

    result[r_h_mask] = r_h[r_h_mask]

    return result


def spiky_derivative(x, y, z, h):
    r = np.sqrt(x**2 + y**2 + z**2)

    n = np.zeros(shape=r.shape)

    r_h = -45 * r * (h - r) ** 2 / (np.pi * h**6)
    r_h_mask = r <= h

    n[r_h_mask] = r_h[r_h_mask]

    np.fill_diagonal(r, 1)

    wx = n * x / r
    wy = n * y / r
    wz = n * z / r

    return wx, wy, wz

In [22]:
N = 800  # Number of particles
pos = np.random.randn(N, 3)
# vel = np.zeros(pos.shape)
vel = np.random.randn(N, 3)

pos, vel = run_simulation(
    "spiky",
    pos,
    vel,
    t=0,  # current time of the simulation
    tEnd=12,  # time at which simulation ends
    dt=0.04,  # timestep
    M=10,  # star mass
    R=1,  # star radius
    h=0.1,  # smoothing length
    k=0.1,  # equation of state constant
    n=0.75,  # polytropic index
    nu=2,  # damping
    kernel=spiky,
    grad_kernel=spiky_derivative,
)

100%|██████████| 300/300 [00:04<00:00, 60.89it/s]


In [23]:
pos2 = np.concatenate([pos + (1, 1, 0), pos + (-1, -1, 0)])
vel2 = np.concatenate([vel + (-3, -1, 0), vel + (3, 1, 0)])

pos2 += np.random.normal(size=pos2.shape) * 0.01
vel2 += np.random.normal(size=pos2.shape) * 0.1

pos3, vel3 = run_simulation(
    "spiky_2_stars",
    pos2,
    vel2,
    t=0,  # current time of the simulation
    tEnd=24,  # time at which simulation ends
    dt=0.04,  # timestep
    M=3,  # star mass
    R=1,  # star radius
    h=0.1,  # smoothing length
    k=0.1,  # equation of state constant
    n=0.75,  # polytropic index
    nu=2,  # damping
    kernel=spiky,
    grad_kernel=spiky_derivative,
)

100%|██████████| 600/600 [00:13<00:00, 46.03it/s]


The use of spiky kernel results in a much different simulation. It seems that the objects don't change their density and move slowly and steadily